In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

# Red neuronal secuencial
from tensorflow.keras.models import Sequential
# Capa completamente conectada
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D
# Optimizador
from tensorflow.keras.optimizers import Adam

In [2]:
#Leer el dataset 
df = pd.read_csv('df2_completado - df.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CODIGO      1454 non-null   int64  
 1   GENERO      1454 non-null   object 
 2   ESTUDIANTE  1454 non-null   object 
 3   FACULTAD    1454 non-null   object 
 4   PROGRAMA    1454 non-null   object 
 5   SEDE        1454 non-null   object 
 6   JORNADA     1454 non-null   object 
 7   SEMESTRE    1454 non-null   int64  
 8   PERIODO     1454 non-null   float64
dtypes: float64(1), int64(2), object(6)
memory usage: 102.4+ KB


In [3]:
df.head()

,CODIGO,GENERO,ESTUDIANTE,FACULTAD,PROGRAMA,SEDE,JORNADA,SEMESTRE,PERIODO
0,100423020941,M,Regular,Facultad de Ciencias Agrarias,Ingenieria Agroindustrial Diurno - Sede Santan...,SANTANDER DE QUILICHAO,DIURNO,2,2023.2
1,100423020942,M,Regular,Facultad de Ciencias Agrarias,Ingenieria Agroindustrial Diurno - Sede Santan...,SANTANDER DE QUILICHAO,DIURNO,1,2023.2
2,100423020943,F,Regular,Facultad de Ciencias Agrarias,Ingenieria Agroindustrial Diurno - Sede Santan...,SANTANDER DE QUILICHAO,DIURNO,1,2023.2
3,100423020944,M,Regular,Facultad de Ciencias Agrarias,Ingenieria Agroindustrial Diurno - Sede Santan...,SANTANDER DE QUILICHAO,DIURNO,1,2023.2
4,100423020945,M,Regular,Facultad de Ciencias Agrarias,Ingenieria Agroindustrial Diurno - Sede Santan...,SANTANDER DE QUILICHAO,DIURNO,2,2023.2


Verificamos si la base de datos tiene archivos o valores faltantes

In [4]:
missing = df.isnull().any()

print("¿Columnas con datos faltantes?")
print(missing)

¿Columnas con datos faltantes?
CODIGO        False
GENERO        False
ESTUDIANTE    False
FACULTAD      False
PROGRAMA      False
SEDE          False
JORNADA       False
SEMESTRE      False
PERIODO       False
dtype: bool


In [5]:
df["GENERO"] = df["GENERO"].astype("category")
df["ESTUDIANTE"] = df["ESTUDIANTE"].astype("category")
df["FACULTAD"] = df["FACULTAD"].astype("category")
df["PROGRAMA"] = df["PROGRAMA"].astype("category")
df["SEDE"] = df["SEDE"].astype("category")
df["JORNADA"] = df["JORNADA"].astype("category")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   CODIGO      1454 non-null   int64   
 1   GENERO      1454 non-null   category
 2   ESTUDIANTE  1454 non-null   category
 3   FACULTAD    1454 non-null   category
 4   PROGRAMA    1454 non-null   category
 5   SEDE        1454 non-null   category
 6   JORNADA     1454 non-null   category
 7   SEMESTRE    1454 non-null   int64   
 8   PERIODO     1454 non-null   float64 
dtypes: category(6), float64(1), int64(2)
memory usage: 44.9 KB


CODIFICAMOS LOS DATOS UTILIZANDO LA CLASE OneHontEnconder de la libreria sklearn

In [7]:
# 5. Definir columnas categóricas para codificar
columnas_categoricas = ["GENERO", "ESTUDIANTE", "FACULTAD", "PROGRAMA", "SEDE", "JORNADA"]

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
# 6. Inicializar codificador
codificador = OneHotEncoder (sparse_output=False)  # Usa 'sparse=False' si usas versión antigua de sklearn

In [10]:
# 7. Aplicar codificación
codificacion_array = codificador.fit_transform(df[columnas_categoricas])

In [11]:
# 8. Crear DataFrame con nombres de columnas codificadas
codificador_cols = pd.DataFrame(codificacion_array, columns=codificador.get_feature_names_out(columnas_categoricas), index=df.index)

In [12]:
# 9. Reemplazar columnas originales por las codificadas
df = df.drop(columns=columnas_categoricas)
df = pd.concat([df, codificador_cols], axis=1)

In [13]:
# 10. Guardar el nuevo dataset
df.to_csv("df_codificado_final.csv", index=False)

In [14]:
# 11. Mostrar las primeras filas
print("\nDataFrame final con variables numéricas:")
print(df.head())


DataFrame final con variables numéricas:
         CODIGO  SEMESTRE  PERIODO  GENERO_F  GENERO_M  ESTUDIANTE_Primiparo  \
0  100423020941         2   2023.2       0.0       1.0                   0.0   
1  100423020942         1   2023.2       0.0       1.0                   0.0   
2  100423020943         1   2023.2       1.0       0.0                   0.0   
3  100423020944         1   2023.2       0.0       1.0                   0.0   
4  100423020945         2   2023.2       0.0       1.0                   0.0   

   ESTUDIANTE_Regular  FACULTAD_Facultad de Artes  \
0                 1.0                         0.0   
1                 1.0                         0.0   
2                 1.0                         0.0   
3                 1.0                         0.0   
4                 1.0                         0.0   

   FACULTAD_Facultad de Ciencias Agrarias  \
0                                     1.0   
1                                     1.0   
2                      

In [17]:
# Leer los archivos CSV
X_encoded = pd.read_csv("df_codificado_final.csv")  # Reemplaza con el nombre real del archivo que contiene X
y = pd.read_csv("df_salida_final.csv")          # Reemplaza con el nombre real del archivo que contiene y

# Dividir los datos en 70% entrenamiento y 30% prueba
train_size = int(0.7 * len(X_encoded))

X_train = X_encoded.iloc[:train_size]
X_test = X_encoded.iloc[train_size:]

y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]

# Guardar los archivos CSV
X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)